In [1]:
# %% 
# import packages
import os
import sys
import timeit
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

sys.path.insert(0,'C:\\Codes\\github\\signal_processing_dev\\dstMain_reference_model')

from preset_parser import config_load_preset, config_para_parse, config_para_process
from rf_data_op import rf_data_read,rf_data_read_npy,rf_data_remap, rf_data_bandpass,rf_data_interp,rf_data_append, rf_data_apod, rf_data_diag, rf_data_plot
from beamformer_calc_rtbf import recon_compute_geometry, recon_field_pre_compute, recon_process_rtbf
from env_data_op import env_data_get, env_data_compress, env_qbp_filter,env_data_interpolate
from img_data_op import image_conversion, image_polar,image_crop, image_plot_linear, image_plot_curvelinear, image_interpolate, image_LUT, image_contrast, image_sharpness

from beamformer_calc_rtbf_GPU import recon_process_rtbf

In [2]:
def read_data_and_configs(raw_rf_path, config_path):
    preset = config_load_preset(config_path)

    # rf_data = rf_data_read(file_path) # use this to read bin files
    rf_data = rf_data_read_npy(raw_rf_path, file_name=None)  # use this to read npy files
    rf_data = rf_data_remap(rf_data, list(preset["beamformer"].config_set[0].pic_sub_array.channel_map))
    # rf_data_plot(rf_data_diag(rf_data), loc_plot=64,fig_title='P/E image')# plot the rf P/E for inspection

    # interpolates RF data in the time direction. NOTE: interpolation factor should be a parameter read from tableInputs.pbtx
    # rf_data = rf_data_interp(rf_data,preset['table_inputs'].rx_mrf_decim_rate[0])
    # data apodization, zeros the last two points for signal artifact removal
    rf_data = rf_data_apod(rf_data, 2, loc=0)
    # data appending, add two zeros at the end.  Beamformer will use this sample when delay and sum points beamformer to out of range data
    rf_data = rf_data_append(rf_data, 2, loc=0)

    # reformat config into
    # xdc:  transducer geometries
    # field:  acoustic field parameters
    # param:  other miscellaneous parameters
    xdc, field, param = config_para_parse(preset, rf_data.shape)
    field["line_length"] = 0.12
    xdc, field, param = config_para_process(xdc, field, param)
    xdc[
        "xmit_focus"] = 0.050  # 0.03494 # NOTE : currently manual definition of xdc focus since the're no way to pull focus information except from DSTMain screenshot

    return rf_data, xdc, field, param

In [3]:
print("grabbing data and config")

Tim = True

if not Tim:

    # path and presets
    config_dir = 'C:\\Codes\\github\\signal_processing_dev\\SK_reference_model\\presets'#
    file_path = 'C:\\Users\\dst01\\OneDrive - deepsightinc.com\\work\\Projects\\Mixed Array Transducer\\20241016 harmonic imaging\\pic_mrf2-10_3.05mhz_2cyc_20hv_wfm3_inv'
    file_name = "_PIC_b_mode_0_raw_rf"

    probe_dir = 'DL3-14'
    config_path = os.path.join(config_dir,probe_dir)

    preset_name = 'DL3-14_PIC'

else:
    data_dir =  'Challenge_sets/L7-4_General'
    config_path = os.path.join(data_dir, 'presets/scratch')

    raw_rf_dir = os.path.join(data_dir, 'data')

    file_name = 'L7-4_General_b_mode_0_raw_rf.npy'
    raw_rf_path = os.path.join(raw_rf_dir, file_name)


grabbing data and config


In [4]:
rf_data, xdc, field, param = read_data_and_configs(raw_rf_path, config_path)

loading preset Challenge_sets/L7-4_General\presets/scratch


In [5]:
print("processing reconstruction")

# generate calculated parameters and add to existing config stuctures, params broken out to facilitate parameter optimization
# pre compute reconstruction field geometry and limiters
print('precomputing geometry...')
recon = recon_compute_geometry(xdc, field, param)  # pre-compute reconstruction field
print('precomputing field...')
recon = recon_field_pre_compute(recon, xdc, field, param)

processing reconstruction
precomputing geometry...
precomputing field...


In [68]:
print('running GPU beamformer')
"""
calculates the delay, linearize operation and perform lookup
Parameters
----------
rf_data, recon, field, param

Returns : recon_image_sum
-------
"""
# convert numpy arrays to cupy arrays
#recon["tx_limiter"] = recon["tx_limiter"].get()
import cupy as np
rf_data = np.asarray(rf_data)

# process beamforming
c_inv = 1/field["c"]
# calculate delay from transmit beam center to each virtual point source
T_vps = xdc["xmit_focus"] * c_inv + param["tx_delay"] 
# store data into linear matrix, matrix shape: ( (param['recon_num_depth'])* (field["channels"]) *(field["lines"]) ) * 3
if param["no_tx_limiter"] == 1: 
    recon["tx_limiter"] = np.ones(recon["tx_limiter"].shape)
    
# tiling coordinates to create meshgrid and then flatten
p_vps_linear = np.reshape(np.tile(recon["vps_grid_3D"],(1,field["channels"]*param['recon_num_depth'])),
                            [(field["lines"]) * (field["channels"]) * (param['recon_num_depth']) ,3])
p_rcv_linear = np.reshape(np.tile(recon["rcv_xdc_grid_3D"],(field["lines"],1,param['recon_num_depth'])),
                            [(field["lines"]) * (field["channels"]) * (param['recon_num_depth']) ,3])
p_pixel_linear=np.reshape(np.tile(recon["px_grid_3D"],(field["channels"],1)),
                        [(field["lines"])*(field["channels"])*(param['recon_num_depth']),3])
T_vps_linear = np.tile(T_vps,(field["lines"]*field["channels"]*param['recon_num_depth'])) 

# calculate vectors from virtual point source to pixel locations
P_p_to_vps_linear = p_pixel_linear - p_vps_linear
# calculate vectors from pixel locations to receive elements
P_p_to_rcv_linear = p_pixel_linear - p_rcv_linear
# calculate delays from pixel to virtual point source 
T_p_to_vps_linear = np.linalg.norm(P_p_to_vps_linear, axis=1)*c_inv 
# calculate delays from pixel to channel elements 
T_p_to_rcv_linear = np.linalg.norm(P_p_to_rcv_linear, axis=1)*c_inv

# flatten limiters
# rcv_limiter_linear = np.reshape(recon["rcv_limiter"],[(field["channels"])*(field["lines"])*(param['recon_num_depth'])])
rcv_norm_linear = 1/np.reshape(np.tile(np.sum(recon["rcv_limiter"],axis=0)+1e-12,(field["lines"])),\
                                [(field["channels"])*(field["lines"])*(param['recon_num_depth'])])
recon_apod_linear = np.reshape(np.asarray(recon["apod_window"]),[(field["channels"])*(field["lines"])* (param['recon_num_depth'])])

# RTBF line of interest for each transmit, you can set it to zero to manually disable rtbf
loop_range = np.arange(-int(param["recon_tx_width_wavelength"]/2),int(param["recon_tx_width_wavelength"]/2)+1).get()

running GPU beamformer


In [69]:
if len(loop_range) == 1:
    tx_norm = 1/(recon["tx_limiter"][63,63+loop_range[0],:]+1e-12)
else:
    print(type(recon["tx_limiter"]))
    tx_norm = 1/(np.sum(recon["tx_limiter"][63,63+loop_range,:],axis=0)+1e-12)

<class 'numpy.ndarray'>


In [70]:
tx_norm = (tx_norm/np.min(tx_norm)).astype(int)
tx_norm_linear = np.tile(tx_norm,(field["lines"]*field["channels"]))
norm_linear = rcv_norm_linear*recon_apod_linear*tx_norm_linear

In [71]:
norm_linear = np.where(norm_linear>20, 0 , norm_linear) 

if param["no_rcv_limiter"] == 1: 
    norm_linear = np.ones(norm_linear.shape)   

In [72]:


for i in loop_range:# loop through rtbf lines
    tx_limiter_linear = np.tile(recon["tx_limiter"][63,63+i,:], [(field["lines"])*(field["channels"])])
    # sum all delays
    delay_sample_linear = np.round((T_p_to_rcv_linear * np.sign(P_p_to_rcv_linear[:,0]) * int(not(param["no_rcv_beamforming"]))
                                    + np.roll(T_p_to_vps_linear,i*field["channels"]*param['recon_num_depth'],axis=0)*
                                    np.roll(np.sign(P_p_to_vps_linear[:,0]),i*field["channels"]*param['recon_num_depth'],axis=0)
                                    + T_vps_linear)*field["fs"]).astype("int")
    # range gate,  this will use the last sample (which is set to zero) when out of data range
    delay_sample_linear = np.where ((delay_sample_linear > rf_data.shape[1]-1) | (delay_sample_linear < 0), rf_data.shape[1]-1, delay_sample_linear)

    # helper functions to index RF data
    index_0_linear = (((np.arange(len(tx_norm_linear))/param['recon_num_depth']).astype("int")) % field["channels"]).astype("int") # channels
    index_1_linear = (np.arange(len(tx_norm_linear))/(field["lines"])/(param['recon_num_depth'])).astype("int") # lines
    
    # lookup appropriate data for the calculated delay
    recon_image_linear= rf_data[index_0_linear,delay_sample_linear,np.roll(index_1_linear,i*field["channels"]*param['recon_num_depth'])] * norm_linear *tx_limiter_linear #conduct lookup with apodization
    # reshape for 3d data output
    recon_image = np.reshape(recon_image_linear,[(field["lines"]),(field["channels"]),(param['recon_num_depth'])])
    
    # zeros the pixels with looparound VPS locations
    if i > 0:
        recon_image[0:i,:,:] = 0 
    elif i < 0:
        recon_image[i:(field["channels"]),:,:] = 0 
    
    # sum over the RTBF resonctruction
    if i == loop_range[0]:
        recon_image_log = recon_image
    else:
        recon_image_log = recon_image_log + recon_image

recon_image_sum = recon_image_log/len(loop_range)+1e-9

In [73]:
scale = 7
run_image_processing(recon_image, xdc, field, param, num_samples, scale = scale)

NameError: name 'run_image_processing' is not defined